In [ ]:
# importar la spark session
from pyspark.sql import SparkSession

# crar sessio.
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [40]:
campaign_df = spark.read.csv("./inputs/campanas.csv", header=True, inferSchema=True)
campaign_df.show(truncate=False)

+----------+-----------------------------+
|ID_Campaña|Nombre_Campaña               |
+----------+-----------------------------+
|1001      |Campaña-1-Marca B-Lanzamiento|
|1002      |Campaña-2-Marca A-Lanzamiento|
|1003      |Campaña-3-Marca A-Conciencia |
|1004      |Campaña-4-Marca A-Conciencia |
|1005      |Campaña-5-Marca C-Promoción  |
+----------+-----------------------------+



In [41]:
from pyspark.sql.functions import split, col, concat, lit
# SPLIT Nombre_Campana in 4 columns splited by - using split function and col function
campaign_df = campaign_df.withColumn("prefijo", split(campaign_df["Nombre_Campaña"], "-")[0])
campaign_df = campaign_df.withColumn("id", split(campaign_df["Nombre_Campaña"], "-")[1])
campaign_df = campaign_df.withColumn("marca", split(campaign_df["Nombre_Campaña"], "-")[2])
campaign_df = campaign_df.withColumn("tipo", split(campaign_df["Nombre_Campaña"], "-")[3])
# Create column campaign_name concatenating column prefijo + "_" + column id
campaign_df = campaign_df.withColumn("campaign_name", concat(col("prefijo"), lit("_") , col("id")))
campaign_df =  campaign_df.drop("Nombre_Campaña").drop("prefijo")
campaign_df = campaign_df.withColumnRenamed("ID_Campaña", "campaign_id")
# campaign_df = campaign_df.withColumn("campaign_name", campaign_df["prefijo"] + "_" + campaign_df["id"])
campaign_df.show(truncate=False)

+-----------+---+-------+-----------+-------------+
|campaign_id|id |marca  |tipo       |campaign_name|
+-----------+---+-------+-----------+-------------+
|1001       |1  |Marca B|Lanzamiento|Campaña_1    |
|1002       |2  |Marca A|Lanzamiento|Campaña_2    |
|1003       |3  |Marca A|Conciencia |Campaña_3    |
|1004       |4  |Marca A|Conciencia |Campaña_4    |
|1005       |5  |Marca C|Promoción  |Campaña_5    |
+-----------+---+-------+-----------+-------------+



In [43]:
campaign_df.printSchema()

root
 |-- campaign_id: integer (nullable = true)
 |-- id: string (nullable = true)
 |-- marca: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- campaign_name: string (nullable = true)



In [18]:
group_df = spark.read.csv("./inputs/grupos_anuncios.csv", header=True, inferSchema=True)
group_df.show(truncate=False)

+-----------------+----------+---------------------+
|ID_Grupo_Anuncios|ID_Campaña|Nombre_Grupo_Anuncios|
+-----------------+----------+---------------------+
|2001             |1002      |Grupo-1-Jovenes      |
|2002             |1002      |Grupo-2-Mujeres      |
|2003             |1004      |Grupo-3-Mujeres      |
|2004             |1005      |Grupo-4-Mujeres      |
|2005             |1002      |Grupo-5-Mujeres      |
|2006             |1004      |Grupo-6-Mujeres      |
|2007             |1001      |Grupo-7-Mujeres      |
|2008             |1004      |Grupo-8-Mujeres      |
|2009             |1005      |Grupo-9-Mujeres      |
|2010             |1003      |Grupo-10-Hombres     |
|2011             |1001      |Grupo-11-Jovenes     |
|2012             |1002      |Grupo-12-Jovenes     |
|2013             |1001      |Grupo-13-Jovenes     |
|2014             |1003      |Grupo-14-Mujeres     |
|2015             |1001      |Grupo-15-Hombres     |
+-----------------+----------+----------------

In [42]:
group_df.printSchema()

root
 |-- ID_Grupo_Anuncios: integer (nullable = true)
 |-- ID_Campaña: integer (nullable = true)
 |-- Nombre_Grupo_Anuncios: string (nullable = true)



In [45]:
group_df = group_df.withColumnRenamed("ID_Grupo_Anuncio", "ad_group_id")
group_df = group_df.withColumnRenamed("ID_Campaña", "campaign_id")
group_df = group_df.withColumn("id", split(group_df["Nombre_Grupo_Anuncios"], "-")[1])
group_df = group_df.withColumn("audience", split(group_df["Nombre_Grupo_Anuncios"], "-")[2])
group_df = group_df.withColumn("prefijo", split(group_df["Nombre_Grupo_Anuncios"], "-")[0])
group_df = group_df.withColumn("ad_group_name", concat(col("prefijo"), lit("_"), col("id")))
group_df = group_df.drop("Nombre_Grupo_Anuncios").drop("prefijo")
group_df.show(truncate=False)

+-----------------+-----------+---+--------+-------------+
|ID_Grupo_Anuncios|campaign_id|id |audience|ad_group_name|
+-----------------+-----------+---+--------+-------------+
|2001             |1002       |1  |Jovenes |Grupo_1      |
|2002             |1002       |2  |Mujeres |Grupo_2      |
|2003             |1004       |3  |Mujeres |Grupo_3      |
|2004             |1005       |4  |Mujeres |Grupo_4      |
|2005             |1002       |5  |Mujeres |Grupo_5      |
|2006             |1004       |6  |Mujeres |Grupo_6      |
|2007             |1001       |7  |Mujeres |Grupo_7      |
|2008             |1004       |8  |Mujeres |Grupo_8      |
|2009             |1005       |9  |Mujeres |Grupo_9      |
|2010             |1003       |10 |Hombres |Grupo_10     |
|2011             |1001       |11 |Jovenes |Grupo_11     |
|2012             |1002       |12 |Jovenes |Grupo_12     |
|2013             |1001       |13 |Jovenes |Grupo_13     |
|2014             |1003       |14 |Mujeres |Grupo_14    